# ChatBot to Provide Flight Details from Conversational Text

## Steps

* Import necessary libraries
* Load raw data in a json format (intents.json)
* Preprocess raw data to get training data
* Train data using tflearn DNN model, and save model
* Create a function to take input texts and return output texts based on model predictions of trained responses
* Start talking with the Bot

### Import Necessary Libraries

In [4]:
import warnings
warnings.filterwarnings("ignore")

import nltk
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer()
import numpy as np
import pandas as pd
import tflearn
import tensorflow
from tensorflow import keras
import random
import json
import pickle
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()

nltk.download('punkt')
nltk.download('wordnet')


from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Dropout
from tensorflow.keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /home/learner/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/learner/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


### Load Raw Data in .json Format (intents.json)

In [11]:
with open("intents_2.json") as file:
    data = json.load(file)

### Preprocess Raw Data to Get Training Data

In [12]:
words = []
labels = []
docs_x = []
docs_y = []

for intent in data['intents']:
    for pattern in intent['patterns']:
        wrds = nltk.word_tokenize(pattern)
        words.extend(wrds)
        docs_x.append(wrds)
        docs_y.append(intent['tag'])
    if intent['tag'] not in labels:
        labels.append(intent['tag'])
        
words = [stemmer.stem(w.lower()) for w in words if w not in ['?', '!']]
words = sorted(list(set(words)))
labels = sorted(labels)

training = []
output = []

out_empty = [0 for _ in range(len(labels))]

for x, doc in enumerate(docs_x):
    bag = []

    wrds_doc = [stemmer.stem(wd) for wd in doc]

    for _ in words:
        if _ in wrds_doc:
            bag.append(1)
        else:
            bag.append(0)

    output_row = out_empty[:]
    output_row[labels.index(docs_y[x])] = 1

    training.append(bag)
    output.append(output_row)


training = np.array(training)
output = np.array(output)

### Train Data Using tflearn DNN Model, and Save Model

In [13]:
tensorflow.reset_default_graph()

net = tflearn.input_data(shape=[None, len(training[0])])
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, 8)
net = tflearn.fully_connected(net, len(output[0]), activation="softmax")
net = tflearn.regression(net)

model = tflearn.DNN(net)

model.fit(training, output, n_epoch=150, batch_size=8, show_metric=True)
model.save("model.ftlearn")

Training Step: 899  | total loss: 0.12287 | time: 0.027s
| Adam | epoch: 150 | loss: 0.12287 - acc: 1.0000 -- iter: 40/44
Training Step: 900  | total loss: 0.12283 | time: 0.034s
| Adam | epoch: 150 | loss: 0.12283 - acc: 1.0000 -- iter: 44/44
--
INFO:tensorflow:/home/learner/Documents/unit6_project/model.ftlearn is not in all_model_checkpoint_paths. Manually adding it.


### Create Functions 

In [14]:
## Create function to take input texts and return output texts based on model predictions of trained responses

def bag_of_words(s, words):
    bag = [0 for _ in range(len(words))]
    
    s_words = nltk.word_tokenize(s)
    s_words = [stemmer.stem(word.lower()) for word in s_words]
    
    for se in s_words:
        for i, wd in enumerate(words):
            if wd == se:
                bag[i] = 1
    return np.array(bag)

def chat():
    print("Start talking with the bot (type quit to stop)")
    while True:
        inp = input("You: ")
        if inp.lower() == "quit":
            break
        results = model.predict([bag_of_words(inp, words)])
        results_index = np.argmax(results)
        tag = labels[results_index]
        
        for tg in data['intents']:
            if tg['tag'] == tag:
                responses = tg['responses']
        
        print(f'Bot: {random.choice(responses)}')

### Start Chatting with Bot

In [ ]:
chat()

Start talking with the bot (type quit to stop)
You: want to make a trip
Bot: Great!, what are your start and stop locations
You: toronto
Bot: Thank you
You: london
Bot: see you later
You: vancouver
Bot: how can i help you
You: nnn
Bot: Good to see you again
You: Lon
Bot: Hi there
You: want to make a trip
Bot: Great!, what are your start and stop locations
You: toronto
Bot: Thank you
